## Selective Search approach exploration

In [1]:
IMAGE_DIMS = (32,32,3)
INPUT_DIR = '../../input/kaggle-sea-lion/'
INPUT_DATASET_NAME = '02/lion-patches-0px'

In [6]:
%matplotlib inline
import cv2
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt
import sklearn
import os
import glob
import selectivesearch

from modules.logging import logger
import modules.utils as utils
from modules.utils import Timer
import modules.logging
import modules.cnn as cnn
import modules.lions as lions

### Explore known lions image in order to identify key characteristics

In [ ]:
input_dataset_path = INPUT_DIR + utils.dataset_name(INPUT_DATASET_NAME, IMAGE_DIMS)
samples = None
with h5py.File(input_dataset_path, 'r') as input_h5:
    samples = input_h5['X'][20:40]
    utils.show_images(samples, cols=10, is_bgr=True, size=2)

## Mask image so only desired lion colors remain

In [ ]:
img_file = INPUT_DIR + 'Train/0.jpg'
im0 = cv2.imread(img_file)
im = im0[3000:4000,3000:4000,:]
utils.show_image(im, is_bgr=True)

### Calculage average histogram for random samples

In [ ]:
h5file = h5py.File(input_dataset_path, 'r')
xy_generator = utils.BatchGeneratorXYH5(h5file, start_ratio=0, end_ratio=1)
samples, _ = utils.dump_xy_to_array(xy_generator.flow(), xy_generator.size, x=True, y=False, dtype='uint8', random_skip=0.5)
print(np.shape(samples))
samples_hsv = [cv2.cvtColor(s,cv2.COLOR_BGR2HSV) for s in samples]

### Perform histogram backpropagation

In [ ]:
im_hsv = cv2.cvtColor(im,cv2.COLOR_BGR2HSV)

# calculating object histogram
sample_hist = cv2.calcHist(samples_hsv,[0,1], None, [255,255], [0, IMAGE_DIMS[0], 0, IMAGE_DIMS[1]])
# sample_hist = cv2.normalize(sample_hist,0,255,cv2.NORM_MINMAX)

plt.hist(sample_hist)

#perform back projection of histogram
s = np.shape(im_hsv)
im_back = cv2.calcBackProject([im_hsv],[0,1],sample_hist,[0,s[0],0,s[1]],1)

In [ ]:
# Now convolute with circular disc
disc = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
im_back_filter = cv2.filter2D(im_back,-1,disc)

utils.show_image(im_back)

# threshold and binary AND
_,mask = cv2.threshold(im_back_filter,50,255,cv2.THRESH_BINARY_INV)

mask = cv2.merge((mask,mask,mask))
utils.show_image(mask, size=6, is_bgr=True)

kernel = np.ones((3,3),np.uint8)
mask = cv2.erode(mask,kernel,iterations = 5)
mask = cv2.dilate(mask,kernel,iterations = 12)
utils.show_image(mask, size=6, is_bgr=True)

im_masked = cv2.bitwise_and(im_hsv,mask)
im_masked = cv2.cvtColor(im_masked,cv2.COLOR_HSV2BGR)

utils.show_image(im, size=6, is_bgr=True)
utils.show_image(im_masked, size=14, is_bgr=True)


In [ ]:
# params = cv2.SimpleBlobDetector_Params()
# params.minThreshold = 150;
# params.maxThreshold = 256;
# params.filterByArea = True
# params.minArea = 10
# params.filterByCircularity = False
# params.minCircularity = 0.1
# params.filterByConvexity = True
# params.minConvexity = 0.5
# params.filterByInertia = False
# params.minInertiaRatio = 0.5
# detector = cv2.SimpleBlobDetector_create(params)
# keypoints = detector.detect(im_masked)

# im_keypoints = cv2.drawKeypoints(im, keypoints, np.array([]), (0,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
# utils.show_image(im_keypoints, size=12, is_bgr=True)

### Find candidate areas by Selective Search

In [ ]:
import skimage
regions = skimage.segmentation.felzenszwalb(im, scale=100, sigma=0.5, min_size=10)
utils.show_image(regions, size=6)

In [ ]:
print('selectivesearch')
img_lbl, regions = selectivesearch.selective_search(im_masked, scale=200, sigma=0.8, min_size=200)

print('filtering')
regions2 = []
for a in regions:
    size = a['size']
    r = a['rect']
    regions2 = regions2 + [r]

In [ ]:
print('showing')
im2 = im.copy()
print(len(regions2))
count = 0
for r in regions2:
    x, y, w, h = r
    if(w==0 or h==0):
        continue
    if(20<w<120 and 20<h<120 and 0.7<(w/h)<1.3):
        cv2.rectangle(im2, (x, y), (x+w, y+h), (0,255,0), 2)
        count += 1
print(count)
utils.show_image(im2, is_bgr=True, size=14)